In [31]:
from sklearn.datasets import load_wine
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve

SEED = 42

In [7]:
df = pd.read_csv("classification.csv")
df.head(2)

,true,pred
0,1,0
1,1,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   true    200 non-null    int64
 1   pred    200 non-null    int64
dtypes: int64(2)
memory usage: 3.2 KB


In [45]:
# Чему равны значения TP, FP, FN, TN (укажете четыре числа через пробел)?

def df_cond(cond):
    return df[cond].shape[0]
TP = df_cond((df.true == 1) & (df.pred == 1))
TN = df_cond((df.true == 0) & (df.pred == 0))
FN = df_cond((df.true == 0) & (df.pred == 1))
FP = df_cond((df.true == 1) & (df.pred == 0))

with open("1.txt", 'w') as f:
    print(f"{TP} {FN} {FP} {TN}", file=f)

In [25]:
def calc_metric(df, metric):
    return metric(df.true, df.pred)
acc = calc_metric(df, accuracy_score)
pr = calc_metric(df, precision_score)
recall = calc_metric(df, recall_score)
f1 = calc_metric(df, f1_score)

with open("2.txt", "w") as f:
    print(f"{acc} {pr} {recall} {f1}", file=f)

In [27]:
scores = pd.read_csv("scores.csv")
scores.head(2)

,true,score_logreg,score_svm,score_knn,score_tree
0,0,0.683832,0.145976,0.787063,0.500000
1,1,0.801966,0.239511,1.000000,0.833333


In [42]:
max_sc = 0.
method = ''
for sc in ['score_logreg', 'score_svm', 'score_knn', 'score_tree']:
    sc_num = roc_auc_score(scores.true, scores[sc])
    if sc_num > max_sc:
        method = sc
        max_sc = sc_num
        
with open("3.txt", 'w') as f:
    print(f"{method}", file=f, end='')
!cat 3.txt

score_logreg

In [69]:
max_pr = 0.
method = ''
for sc in ['score_logreg', 'score_svm', 'score_knn', 'score_tree']:
    df = pd.DataFrame(precision_recall_curve(scores.true, scores[sc])).T
    df.columns = ['precision', 'recall', 'threshold']
    display(df.tail())
    pr = df[df.recall >= 0.7]['precision'].max()
    print(pr, sc)
    if pr > max_pr:
        max_pr = pr
        method=sc

        
        
print(max_pr)
with open("4.txt", 'w') as f:
    print(f"{method}", file=f, end='')
!cat 4.txt

,precision,recall,threshold
193,0.75,0.030612,0.961081
194,1.00,0.030612,0.963559
195,1.00,0.020408,0.964515
196,1.00,0.010204,0.973079
197,1.00,0.000000,NaN


0.6302521008403361 score_logreg


,precision,recall,threshold
195,0.750000,0.030612,0.797968
196,0.666667,0.020408,0.830973
197,1.000000,0.020408,0.900264
198,1.000000,0.010204,0.919219
199,1.000000,0.000000,NaN


0.6228070175438597 score_svm


,precision,recall,threshold
100,0.642857,0.367347,0.815816
101,0.636364,0.357143,0.903657
102,0.629630,0.346939,0.944694
103,0.622642,0.336735,1.000000
104,1.000000,0.000000,NaN


0.6065573770491803 score_knn


,precision,recall,threshold
8,0.670732,0.561224,0.700000
9,0.700000,0.428571,0.833333
10,0.717949,0.285714,0.923077
11,0.750000,0.183673,1.000000
12,1.000000,0.000000,NaN


0.6517857142857143 score_tree
0.6517857142857143
score_tree

In [64]:
df = pd.DataFrame(precision_recall_curve(scores.true, scores["score_logreg"])).T
df.columns = ['precision', 'recall', 'threshold']
df[df.recall >= 0.7]['precision'].max()

0.6302521008403361

In [57]:
df = pd.DataFrame(precision_recall_curve(scores.true, scores['score_logreg'])).T
df.columns = ['precision', 'recall', 'threshold']
df

,precision,recall,threshold
0,0.497462,1.000000,0.070739
1,0.494898,0.989796,0.074923
2,0.492308,0.979592,0.086688
3,0.494845,0.979592,0.096733
4,0.497409,0.979592,0.101603
...,...,...,...
193,0.750000,0.030612,0.961081
194,1.000000,0.030612,0.963559
195,1.000000,0.020408,0.964515
196,1.000000,0.010204,0.973079
